# Api Import Donnee

https://drive.google.com/file/d/1nlP-a1_1vNeo43m9doUR4Y6eQ0Zh8th_/view?usp=sharing lien pour le csv

Pour l'importation par API, nous avons choisi DataTourisme pour récupérer un maximum d'événements et de lieux touristiques.
Sur DataTourisme, il y a 400 000 événements/lieux touristiques, chacun contenant de nombreuses informations. Pour notre projet RSE utilisant le NLP, les informations qui nous intéressent le plus sont les descriptions textuelles. Nous avons également récupéré les informations concernant les adresses ainsi que les dates de début et de fin pour les événements temporaires.

Pour ce faire, nous avons d'abord créé un compte gratuitement et demandé une clé API. Ensuite, il faut créer un flux ; pour nous, le flux est composé de tous les événements, mais si, par exemple, nous voulons uniquement récupérer les événements sportifs dans la ville de Paris, nous pouvons déjà appliquer un filtre. Ensuite, nous devons attendre une journée pour que les données soient générées. Grâce à ce code Python, nous pouvons récupérer les informations et les mettre dans un dataset afin de les sauvegarder en .csv.

# Code

## Fonction pour récupérer les adresses

In [3]:
import json
def find_address_by_id(address_id, zip_ref):
    # Parcourir tous les fichiers JSON pour trouver celui qui contient l'ID de l'adresse
    for filename in zip_ref.namelist():
        if filename.endswith('.json') and 'objects' in filename:
            with zip_ref.open(filename) as json_file:
                data = json.loads(json_file.read().decode('utf-8'))
                if data.get('@id') == address_id:
                    # Extraire les informations d'adresse
                    return {
                        'ville': data.get('http://schema.org/addressLocality', [{}])[0].get('@value'),
                        'code_postal': data.get('http://schema.org/postalCode', [{}])[0].get('@value'),
                        'rue': data.get('http://schema.org/streetAddress', [{}])[0].get('@value')
                    }
    return None

## Fonction pour récupérer les évenements / lieux touristiques et code principales

In [7]:
import requests
import json
import zipfile
from io import BytesIO
import pandas as pd

API_KEY = "f6e8553b-0fdb-45b6-becb-ecf54d29c362"
base_url = f"https://diffuseur.datatourisme.fr/webservice/a309f2d6a95e8b424b5495ba9742ffe3/{API_KEY}"

try:
    response = requests.post(base_url, timeout=10)
    
    if response.status_code == 200:
        print("Réponse reçue avec succès")
        zip_content = BytesIO(response.content)
        events_data = []
        
        with zipfile.ZipFile(zip_content, 'r') as zip_ref:
            
            for filename in zip_ref.namelist():
                if filename.endswith('.json') and 'objects' in filename:
                    with zip_ref.open(filename) as json_file:
                        data = json.loads(json_file.read().decode('utf-8'))
                        
                        # Extraire les informations pertinentes
                        event = {
                            'id': data.get('@id', ''),
                            'nom': data.get('rdfs:label', ''),
                            'date_debut': data.get('schema:startDate', ''),
                            'date_fin': data.get('schema:endDate', ''),
                            'description': data.get('rdfs:comment', ''),
                            'derniere_maj': data.get('lastUpdate', ''),
                            'ville': None,
                            'code_postal': None,
                            'rue': None
                        }
                                                
                        location_data = data.get('isLocatedAt', [])
                        if location_data:
                            location = location_data[0]  # Premier élément de la liste
                            if 'schema:address' in location:
                                address_data = location['schema:address'][0]  # Premier élément de la liste d'adresses
                                event.update({
                                    'ville': address_data.get('schema:addressLocality'),
                                    'code_postal': address_data.get('schema:postalCode'),
                                    'rue': address_data.get('schema:streetAddress', [])[0] if address_data.get('schema:streetAddress') else None
                                })
                        events_data.append(event)

        # Créer le DataFrame
        df = pd.DataFrame(events_data)
        print("\nAperçu des données extraites:")
        print(df.head())
        print(f"\nNombre total d'événements: {len(df)}")
        
        # Sauvegarder en CSV
        df.to_csv('evenements_culturels.csv', index=False, encoding='utf-8-sig')
        print("\nLes données ont été sauvegardées dans 'evenements_culturels.csv'")

except Exception as e:
    print(f"Erreur: {str(e)}")
    print(f"Type d'erreur: {type(e)}")

Réponse reçue avec succès

Aperçu des données extraites:
                                                  id  \
0  https://data.datatourisme.fr/10/000283ba-f94b-...   
1  https://data.datatourisme.fr/10/000ce9fa-eaca-...   
2  https://data.datatourisme.fr/10/0016c366-d21e-...   
3  https://data.datatourisme.fr/10/0016e4ef-68b7-...   
4  https://data.datatourisme.fr/10/00206603-d2ce-...   

                                                 nom date_debut date_fin  \
0       {'fr': ['VILLA GALLO-ROMAINE DU GROSSWALD']}                       
1                        {'fr': ['PARC DU TONNEAU']}                       
2  {'fr': ['MEUBLÉ - RÉSIDENCE LES VOSGES - STUDI...                       
3      {'fr': ['ÉGLISE PAROISSIALE DE LA NATIVITÉ']}                       
4                 {'fr': ['RESTAURANT MC DONALD'S']}                       

                                         description derniere_maj  \
0  {'de': ['Die Ruinen der Villa im Grosswald lie...   2024-04-11   
1  {'de': [

## Clean de la données

In [8]:
df = pd.read_csv('evenements_culturels.csv',sep = ',')
df.head(5)

C:\Users\thesy\AppData\Local\Temp\ipykernel_11204\666062775.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('evenements_culturels.csv',sep = ',')


,id,nom,date_debut,date_fin,description,derniere_maj,ville,code_postal,rue
0,https://data.datatourisme.fr/10/000283ba-f94b-...,{'fr': ['VILLA GALLO-ROMAINE DU GROSSWALD']},NaN,NaN,{'de': ['Die Ruinen der Villa im Grosswald lie...,2024-04-11,Sarreinsming,57905,NaN
1,https://data.datatourisme.fr/10/000ce9fa-eaca-...,{'fr': ['PARC DU TONNEAU']},NaN,NaN,{'de': ['Gegenüber dem Waschhaus liegt der Par...,2024-05-22,Vandœuvre-lès-Nancy,54500,Rue du tonneau
2,https://data.datatourisme.fr/10/0016c366-d21e-...,{'fr': ['MEUBLÉ - RÉSIDENCE LES VOSGES - STUDI...,NaN,NaN,{'de': ['Residenz im Stadtzentrum in der Nähe ...,2024-11-08,La Vôge-les-Bains,88240,11/13 rue du Général Leclerc
3,https://data.datatourisme.fr/10/0016e4ef-68b7-...,{'fr': ['ÉGLISE PAROISSIALE DE LA NATIVITÉ']},NaN,NaN,"{'de': ['Die Geburtskirche, die für die Aufnah...",2024-12-06,Sierck-les-Bains,57480,rue de la Tour de l'Horloge
4,https://data.datatourisme.fr/10/00206603-d2ce-...,"{'fr': [""RESTAURANT MC DONALD'S""]}",NaN,NaN,{'de': ['Schnellrestaurants. Im Gewerbegebiet ...,2023-11-21,Verdun,55100,17 Rue Paul Eugène Martin


In [9]:
import ast
from datetime import datetime

# Fonction pour extraire le texte français d'un dictionnaire de langues
def extract_french_text(text):
    try:
        # Convertir la chaîne en dictionnaire si c'est une chaîne
        if isinstance(text, str):
            text_dict = ast.literal_eval(text)
        else:
            text_dict = text
            
        # Retourner le texte français s'il existe
        if 'fr' in text_dict:
            return text_dict['fr'][0] if isinstance(text_dict['fr'], list) else text_dict['fr']
        return None
    except:
        return None

# Fonction pour extraire les dates
def extract_dates(date_str):
    try:
        # Convertir la chaîne en liste si c'est une chaîne
        if isinstance(date_str, str):
            dates = ast.literal_eval(date_str) if '[' in date_str else [date_str]
        else:
            dates = date_str
            
        # Convertir les dates en objets datetime
        dates = [datetime.strptime(d, '%Y-%m-%d') for d in dates if d]
        
        if dates:
            return min(dates).strftime('%Y-%m-%d'), max(dates).strftime('%Y-%m-%d')
        return None, None
    except:
        return None, None

# Créer le nouveau DataFrame nettoyé
df_clean = df.copy()

# Nettoyer les colonnes nom et description
df_clean['nom'] = df_clean['nom'].apply(extract_french_text)
df_clean['description'] = df_clean['description'].apply(extract_french_text)

# Nettoyer les dates
df_clean[['date_debut', 'date_fin']] = pd.DataFrame(
    df_clean['date_debut'].apply(extract_dates).tolist(),
    index=df_clean.index
)

# Afficher le résultat
df_clean.head()

,id,nom,date_debut,date_fin,description,derniere_maj,ville,code_postal,rue
0,https://data.datatourisme.fr/10/000283ba-f94b-...,VILLA GALLO-ROMAINE DU GROSSWALD,None,None,Les ruines de la Villa du Grosswald sont en bo...,2024-04-11,Sarreinsming,57905,NaN
1,https://data.datatourisme.fr/10/000ce9fa-eaca-...,PARC DU TONNEAU,None,None,"En face du lavoir, le parc du Tonneau, situé d...",2024-05-22,Vandœuvre-lès-Nancy,54500,Rue du tonneau
2,https://data.datatourisme.fr/10/0016c366-d21e-...,MEUBLÉ - RÉSIDENCE LES VOSGES - STUDIO N95,None,None,Résidence située en centre-ville à proximité d...,2024-11-08,La Vôge-les-Bains,88240,11/13 rue du Général Leclerc
3,https://data.datatourisme.fr/10/0016e4ef-68b7-...,ÉGLISE PAROISSIALE DE LA NATIVITÉ,None,None,Destinée à recevoir les Chevaliers de l'Ordre ...,2024-12-06,Sierck-les-Bains,57480,rue de la Tour de l'Horloge
4,https://data.datatourisme.fr/10/00206603-d2ce-...,RESTAURANT MC DONALD'S,None,None,Restauration rapide. Situé sur la zone commerc...,2023-11-21,Verdun,55100,17 Rue Paul Eugène Martin


In [10]:
line_1 = df_clean.iloc[0]
line_1['description']

"Les ruines de la Villa du Grosswald sont en bordure d'un plateau calcaire et dominent la vallée du Fusslach. La villa romaine s'insérait dans un environnement complexe, à proximité d'une voie, d'une source, d'autres villas et d'un vicus."